# 1. Install and Import Dependencies

In [ ]:
pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Clonning YoloV5


In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!cd yolov5 && pip install -r requirements.txt

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# 2. Load Model

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [ ]:
model

# 3. Make Detections with a Sample Image


In [ ]:
# img = 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Cars_in_traffic_in_Auckland%2C_New_Zealand_-_copyright-free_photo_released_to_public_domain.jpg/800px-Cars_in_traffic_in_Auckland%2C_New_Zealand_-_copyright-free_photo_released_to_public_domain.jpg'
# img = 'https://ultralytics.com/images/zidane.jpg'
img = 'traffic-jamz-2.jpg'

In [ ]:
results = model(img)
results.print()

In [ ]:
%matplotlib inline
plt.imshow(np.squeeze(results.render()))
plt.show()

# Splitting dataset

In [ ]:
!cd yolov5 && \
 python train.py \
   --img 640 \
   --batch 16 \
   --epochs 50 \
   --data /content/drive/MyDrive/DDD-Dataset/sorted/drowsiness.yaml \
   --weights yolov5s.pt \
   --cache

In [ ]:
!cd yolov5 && cp -r runs/train/exp*/ /content/drive/MyDrive/DDD-Dataset/

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train

# 5. Train from scratch

In [ ]:
import uuid   # Unique identifier
import os
import time

In [ ]:
IMAGES_PATH = os.path.join('data', 'images') #/data/images
labels = ['awake', 'drowsy']
number_imgs = 5

In [ ]:
cap = cv2.VideoCapture(0)
# Loop through labels
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(5)

    # Loop through image range
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))

        # Webcam feed
        ret, frame = cap.read()

        # Naming out image path
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')

        # Writes out image to file
        cv2.imwrite(imgname, frame)

        # Render to the screen
        cv2.imshow('Image Collection', frame)

        # 2 second delay between captures
        time.sleep(2)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(os.path.join(IMAGES_PATH, labels[0]+'.'+str(uuid.uuid1())+'.jpg'))

In [ ]:
for label in labels:
    print('Collecting images for {}'.format(label))
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        print(imgname)

In [ ]:
!git clone https://github.com/tzutalin/labelImg

In [ ]:
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
!cd yolov5 && python train.py --img 320 --batch 16 --epochs 500 --data dataset.yml --weights yolov5s.pt --workers 2

# 6. Load Custom Model

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp15/weights/last.pt', force_reload=True)

In [ ]:
img = os.path.join('data', 'images', 'awake.c9a24d48-e1f6-11eb-bbef-5cf3709bbcc6.jpg')

In [ ]:
results = model(img)

In [ ]:
results.print()

In [ ]:
%matplotlib inline
plt.imshow(np.squeeze(results.render()))
plt.show()

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    # Make detections
    results = model(frame)

    cv2.imshow('YOLO', np.squeeze(results.render()))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Preparing the images and Labels, building folder structure

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Checking Paths and Folders

In [ ]:
path_images = '/content/drive/MyDrive/DDD-Dataset/Images'
path_labels = '/content/drive/MyDrive/DDD-Dataset/labels'

# for name in os.listdir(path_images):
#   sub1_path = os.path.join(path_images, name)
#   for people in os.listdir(sub1_path):
#     sub2_path = os.path.join(sub1_path, people)
#     for filename in os.listdir(sub2_path):
#       image_path = os.path.join(sub2_path, filename)
#       print(image_path)

for name in os.listdir(path_labels):
  sub1_path = os.path.join(path_labels, name)
  for people in os.listdir(sub1_path):
    label_path = os.path.join(sub1_path, people)
    for text in os.listdir(label_path):
      text_path = os.path.join(label_path, text)
      with open(text_path, 'r') as file:
        label = file.readlines()[0][0]
        image_name = text
        print(image_name, label)
    # print(sub2_path)
  # print(subfolder_path)


## Train Validation Split


### Train Validation Split

In [ ]:
import os
import random
import shutil

# 1. Configure paths & parameters
base_dir       = '/content/drive/MyDrive/DDD-Dataset/sorted'   # root where 0_awake & 1_sleep live
class_dirs     = ['0_awake', '1_sleep']
train_ratio    = 0.8                                   # 80% train, 20% val

# 2. Define the new folders
img_train_dir  = os.path.join(base_dir, 'images', 'train')
img_val_dir    = os.path.join(base_dir, 'images', 'val')
lbl_train_dir  = os.path.join(base_dir, 'labels', 'train')
lbl_val_dir    = os.path.join(base_dir, 'labels', 'val')

# 3. Create them (if they don't already exist)
for d in (img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir):
    os.makedirs(d, exist_ok=True)

# 4. Loop over each class folder
for cls in class_dirs:
    cls_path = os.path.join(base_dir, cls)
    # collect image files
    files = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg','.png'))]
    random.shuffle(files)

    split_idx   = int(len(files) * train_ratio)
    train_files = files[:split_idx]
    val_files   = files[split_idx:]

    # helper to move images+labels
    def copy_subset(file_list, subset):
        img_dst = img_train_dir  if subset=='train' else img_val_dir
        lbl_dst = lbl_train_dir  if subset=='train' else lbl_val_dir
        for fname in file_list:
            # image
            src_img = os.path.join(cls_path, fname)
            dst_img = os.path.join(img_dst, fname)
            shutil.copy2(src_img, dst_img)

            # corresponding label (.txt)
            txt = os.path.splitext(fname)[0] + '.txt'
            src_lbl = os.path.join(cls_path, txt)
            if os.path.exists(src_lbl):
                dst_lbl = os.path.join(lbl_dst, txt)
                shutil.copy2(src_lbl, dst_lbl)
            else:
                print(f"[WARN] no label for {fname}")

    copy_subset(train_files, 'train')
    copy_subset(val_files,   'val')

print("Done! Your folder tree now looks like:")
!find {os.path.join(base_dir,'images')} -maxdepth 2 -type f | head -n 5; \
 find {os.path.join(base_dir,'labels')} -maxdepth 2 -type f | head -n 5

### Train Test Validation Split

In [ ]:
import os
import random
import shutil
from collections import defaultdict

# 1) Configure paths & split ratios (must sum to 1.0)
base_dir = '/content/drive/MyDrive/DDD-Dataset/sorted'  # Root where 0_awake & 1_sleep live
class_dirs = ['0_awake', '1_sleep']
train_ratio = 0.7  # 70% train
val_ratio = 0.2    # 20% validation
test_ratio = 0.1   # 10% test

# Verify ratios sum to 1
assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Split ratios must sum to 1.0"

# 2) Create directory structure
img_base = os.path.join(base_dir, 'images')
lbl_base = os.path.join(base_dir, 'labels')
for subset in ('train', 'val', 'test'):
    os.makedirs(os.path.join(img_base, subset), exist_ok=True)
    os.makedirs(os.path.join(lbl_base, subset), exist_ok=True)

# 3) Track statistics
stats = defaultdict(lambda: {'images': 0, 'labels': 0, 'missing_labels': 0})

# 4) Process each class
for cls in class_dirs:
    cls_path = os.path.join(base_dir, cls)

    # Verify class directory exists
    if not os.path.exists(cls_path):
        print(f"[ERROR] Class directory {cls_path} does not exist")
        continue

    # Collect image files
    files = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.png'))]
    if not files:
        print(f"[WARN] No image files found in {cls_path}")
        continue

    random.shuffle(files)
    n = len(files)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)
    n_test = n - n_train - n_val  # Ensure all files are used

    splits = {
        'train': files[:n_train],
        'val': files[n_train:n_train+n_val],
        'test': files[n_train+n_val:]
    }

    # Copy images and labels
    for subset, flist in splits.items():
        img_dst = os.path.join(img_base, subset)
        lbl_dst = os.path.join(lbl_base, subset)

        for fname in flist:
            # Copy image
            src_img = os.path.join(cls_path, fname)
            dst_img = os.path.join(img_dst, fname)
            try:
                shutil.copy2(src_img, dst_img)
                stats[subset]['images'] += 1
            except Exception as e:
                print(f"[ERROR] Failed to copy image {src_img}: {e}")
                continue

            # Copy corresponding label
            txt = os.path.splitext(fname)[0] + '.txt'
            src_lbl = os.path.join(cls_path, txt)
            dst_lbl = os.path.join(lbl_dst, txt)
            if os.path.exists(src_lbl):
                try:
                    shutil.copy2(src_lbl, dst_lbl)
                    stats[subset]['labels'] += 1
                except Exception as e:
                    print(f"[ERROR] Failed to copy label {src_lbl}: {e}")
            else:
                print(f"[WARN] No label for {fname} in class {cls}")
                stats[subset]['missing_labels'] += 1

# 5) Print summary and verify folder structure
print("\nDataset Split Summary:")
for subset in ('train', 'val', 'test'):
    print(f"{subset.capitalize()}:")
    print(f"  Images: {stats[subset]['images']}")
    print(f"  Labels: {stats[subset]['labels']}")
    print(f"  Missing Labels: {stats[subset]['missing_labels']}")

print("\nSample of created folder structure:")
for base in (img_base, lbl_base):
    print(f"\n{base}:")
    for subset in ('train', 'val', 'test'):
        subset_path = os.path.join(base, subset)
        if os.path.exists(subset_path):
            files = os.listdir(subset_path)[:5]  # Show first 5 files
            print(f"  {subset}/: {files if files else 'Empty'}")
        else:
            print(f"  {subset}/: Directory does not exist")

## Yolo V9


In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

Loading the model

In [ ]:
from ultralytics import YOLO

# Build a YOLOv9c model from scratch
model = YOLO("yolov9c.yaml")

# Build a YOLOv9c model from pretrained weight
model = YOLO("yolov9c.pt")

# Display model information (optional)
model.info()

YOLOv9c summary: 358 layers, 25,590,912 parameters, 0 gradients, 104.0 GFLOPs


(358, 25590912, 0, 104.02268160000003)

In [ ]:
results = model.train(data="/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness.yaml", epochs=50, imgsz=640)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9c.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, pl

train: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/train.cache... 1125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1125/1125 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.7±0.2 ms, read: 0.4±0.4 MB/s, size: 274.1 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/val.cache... 333 images, 0 backgrounds, 0 corrupt: 100%|██████████| 333/333 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 154 weight(decay=0.0), 161 weight(decay=0.0005), 160 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      10.2G     0.7717     0.3733      1.327          5        640: 100%|██████████| 71/71 [00:33<00:00,  2.14it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  3.72it/s]

                   all        333        333      0.934      0.978      0.986        0.8



50 epochs completed in 0.534 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 51.6MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 51.6MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv9c summary (fused): 156 layers, 25,320,790 parameters, 0 gradients, 102.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  3.55it/s]


                   all        333        333      0.934      0.978      0.986      0.799
                 awake        139        139      0.908      0.986      0.981      0.788
                 sleep        194        194      0.959       0.97      0.991       0.81
Speed: 0.1ms preprocess, 5.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train2


Copying the results to the drive

In [ ]:
cp -r runs/detect/train2*/ /content/drive/MyDrive/DDD-Dataset/results1/

## Yolo v9t


In [ ]:
from ultralytics import YOLO

# Build a YOLOv9c model from scratch
model1 = YOLO("yolov9t.yaml")

# Build a YOLOv9c model from pretrained weight
model1 = YOLO("yolov9t.pt")

# Display model information (optional)
model1.info()

100%|██████████| 4.74M/4.74M [00:00<00:00, 329MB/s]


YOLOv9t summary: 544 layers, 2,128,720 parameters, 0 gradients, 8.5 GFLOPs


(544, 2128720, 0, 8.4827648)

In [ ]:
results1 = model1.train(data="/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness1.yaml", epochs=50, imgsz=640)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness1.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9t.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

train: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/train.cache... 1125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1125/1125 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


WARNING ⚠️ val: Slow image access detected (ping: 21.2±44.0 ms, read: 54.1±59.7 MB/s, size: 274.1 KB). Use local storage instead of remote/mounted storage for better performance. See https://docs.ultralytics.com/guides/model-training-tips/


val: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/val.cache... 333 images, 0 backgrounds, 0 corrupt: 100%|██████████| 333/333 [00:00<?, ?it/s]


...

      50/50      3.43G      0.761     0.3474      1.098          5        640: 100%|██████████| 71/71 [00:15<00:00,  4.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  6.35it/s]

                   all        333        333      0.954      0.941      0.974      0.792



50 epochs completed in 0.246 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 4.6MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 4.6MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv9t summary (fused): 197 layers, 1,971,174 parameters, 0 gradients, 7.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  5.97it/s]


                   all        333        333      0.962      0.938      0.976      0.792
                 awake        139        139      0.956      0.933       0.98      0.788
                 sleep        194        194      0.968      0.943      0.972      0.797
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train3


In [ ]:
cp -r runs/detect/train3*/ /content/drive/MyDrive/DDD-Dataset/Yolo_v9t/

## Yolo_v10n

In [ ]:
from ultralytics import YOLO

# Build a YOLOv9c model from scratch
model_10n = YOLO("yolov10n.yaml")

# Build a YOLOv9c model from pretrained weight
model_10n = YOLO("yolov10n.pt")

# Display model information (optional)
model_10n.info()

100%|██████████| 5.59M/5.59M [00:00<00:00, 363MB/s]


YOLOv10n summary: 223 layers, 2,775,520 parameters, 0 gradients, 8.7 GFLOPs


(223, 2775520, 0, 8.7404288)

In [ ]:
results_10n = model_10n.train(data="/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness1.yaml", epochs=50, imgsz=640)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness1.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

train: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/train.cache... 1125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1125/1125 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 105.3±223.3 ms, read: 42.6±49.9 MB/s, size: 274.1 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/val.cache... 333 images, 0 backgrounds, 0 corrupt: 100%|██████████| 333/333 [00:00<?, ?it/s]


...

      50/50      3.62G      1.531     0.6613      2.109          5        640: 100%|██████████| 71/71 [00:10<00:00,  6.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  8.44it/s]

                   all        333        333      0.933      0.935      0.967      0.778



50 epochs completed in 0.172 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 5.7MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 5.7MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv10n summary (fused): 102 layers, 2,265,558 parameters, 0 gradients, 6.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  6.88it/s]


                   all        333        333      0.927      0.942      0.977      0.779
                 awake        139        139      0.916       0.94      0.977      0.776
                 sleep        194        194      0.938      0.943      0.977      0.782
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train4


In [ ]:
cp -r runs/detect/train4*/ /content/drive/MyDrive/DDD-Dataset/results/Yolo_v10n/

## Yolo v10l

In [ ]:
from ultralytics import YOLO

# Build a YOLOv9c model from scratch
model_10l = YOLO("yolov10l.yaml")

# Build a YOLOv9c model from pretrained weight
model_10l = YOLO("yolov10l.pt")

# Display model information (optional)
model_10l.info()

100%|██████████| 50.0M/50.0M [00:01<00:00, 46.4MB/s]


YOLOv10l summary: 364 layers, 25,888,688 parameters, 0 gradients, 127.9 GFLOPs


(364, 25888688, 0, 127.87471359999999)

In [ ]:
results_10l = model_10l.train(data="/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness1.yaml", epochs=50, imgsz=640)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness1.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

train: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/train.cache... 1125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1125/1125 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 1.1±0.8 ms, read: 44.9±43.9 MB/s, size: 274.1 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/val.cache... 333 images, 0 backgrounds, 0 corrupt: 100%|██████████| 333/333 [00:00<?, ?it/s]


...

      50/50      13.1G      1.568     0.6739       2.77          5        640: 100%|██████████| 71/71 [00:37<00:00,  1.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  3.53it/s]

                   all        333        333      0.911      0.947      0.963       0.77



50 epochs completed in 0.590 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 52.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 52.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv10l summary (fused): 174 layers, 24,310,870 parameters, 0 gradients, 120.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  3.66it/s]


                   all        333        333      0.919      0.932      0.966      0.772
                 awake        139        139      0.908      0.927      0.967      0.771
                 sleep        194        194       0.93      0.938      0.966      0.773
Speed: 0.1ms preprocess, 5.6ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/train5


In [ ]:
cp -r runs/detect/train5*/ /content/drive/MyDrive/DDD-Dataset/Yolo_Results/Yolo_v10l/

## Yolo v11n

In [ ]:
from ultralytics import YOLO

# Build a YOLOv9c model from scratch
model_11n = YOLO("yolo11n.yaml")

# Build a YOLOv9c model from pretrained weight
model_11n = YOLO("yolo11n.pt")

# Display model information (optional)
model_11n.info()

YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(181, 2624080, 0, 6.614336)

In [ ]:
results_11n = model_11n.train(data="/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness1.yaml", epochs=50, imgsz=640)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness1.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

train: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/train.cache... 1125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1125/1125 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 22.0±46.5 ms, read: 38.7±37.8 MB/s, size: 274.1 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/val.cache... 333 images, 0 backgrounds, 0 corrupt: 100%|██████████| 333/333 [00:00<?, ?it/s]


...

      50/50       3.5G     0.7632     0.3494      1.036          5        640: 100%|██████████| 71/71 [00:08<00:00,  8.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  7.68it/s]

                   all        333        333      0.954      0.941      0.981      0.795



50 epochs completed in 0.149 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 5.4MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  6.09it/s]


                   all        333        333       0.94      0.959       0.98      0.795
                 awake        139        139      0.917      0.954      0.974      0.784
                 sleep        194        194      0.962      0.964      0.986      0.806
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train6


In [ ]:
cp -r runs/detect/train6*/ /content/drive/MyDrive/DDD-Dataset/Yolo_Results/Yolo_v11n/

## Yolo v11l

In [ ]:
from ultralytics import YOLO

# Build a YOLOv9c model from scratch
model_11l = YOLO("yolo11l.yaml")

# Build a YOLOv9c model from pretrained weight
model_11l = YOLO("yolo11l.pt")

# Display model information (optional)
model_11l.info()

100%|██████████| 49.0M/49.0M [00:01<00:00, 44.3MB/s]


YOLO11l summary: 357 layers, 25,372,160 parameters, 0 gradients, 87.6 GFLOPs


(357, 25372160, 0, 87.6134912)

In [ ]:
results_11l = model_11l.train(data="/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness1.yaml", epochs=50, imgsz=640)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/DDD-Dataset/sorted/drowsiness1.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train7, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

train: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/train.cache... 1125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1125/1125 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 100.3±221.6 ms, read: 23.5±30.7 MB/s, size: 274.1 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1lHxrXs6MFhuOHACZTt-mRg-_hANl9dEM/DDD-Dataset/sorted/Dataset/labels/val.cache... 333 images, 0 backgrounds, 0 corrupt: 100%|██████████| 333/333 [00:00<?, ?it/s]


...

      50/50      11.2G     0.7862     0.4209      1.185          5        640: 100%|██████████| 71/71 [00:30<00:00,  2.34it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  4.05it/s]

                   all        333        333      0.909      0.975      0.977      0.789



50 epochs completed in 0.482 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 51.2MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 51.2MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11l summary (fused): 190 layers, 25,280,854 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  3.72it/s]


                   all        333        333      0.933      0.959      0.976      0.789
                 awake        139        139      0.918      0.964      0.968      0.786
                 sleep        194        194      0.948      0.954      0.983      0.793
Speed: 0.1ms preprocess, 4.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train7


In [ ]:
cp -r runs/detect/train7*/ /content/drive/MyDrive/DDD-Dataset/Yolo_Results/Yolo_v11l/

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns, matplotlib.pyplot as plt

# Suppose you’ve extracted lists of class IDs for each detected box
# after matching to ground-truth: y_true, y_pred
cm = confusion_matrix(y_true, y_pred, labels=[0,1])
sns.heatmap(cm, annot=True, fmt='d',
            xticklabels=['awake','sleep'],
            yticklabels=['awake','sleep'])
plt.xlabel('True'); plt.ylabel('Predicted'); plt.show()

NameError: name 'y_true' is not defined

In [ ]:
!cd yolov5 && \
 python train.py \
   --img 640 \
   --batch 16 \
   --epochs 50 \
   --data /content/drive/MyDrive/DDD-Dataset/sorted/drowsiness.yaml \
   --weights yolov5s.pt \
   --cache

In [ ]:
import os
import shutil

#–– your two parallel trees:
path_images = '/content/drive/MyDrive/DDD-Dataset/data2/Images'
path_labels = '/content/drive/MyDrive/DDD-Dataset/data2/labels'

#–– where to drop the sorted images:
out_base = '/content/drive/MyDrive/DDD-Dataset/sorted_images'

for dirpath, _, filenames in os.walk(path_labels):
    for fname in filenames:
        if not fname.lower().endswith('.txt'):
            continue

        # 1) read label
        label_file = os.path.join(dirpath, fname)
        with open(label_file, 'r') as f:
            line = f.readline().strip()
            label = line[0] if line else None

        if label not in ('0','1'):
            print(f"⚠️  Skipping {label_file}: bad label “{line}”")
            continue

        # 2) compute relative sub-path under path_labels
        rel_dir = os.path.relpath(dirpath, path_labels)

        # 3) find the image in the parallel tree
        base = os.path.splitext(fname)[0]
        img_dir = os.path.join(path_images, rel_dir)
        img_path = os.path.join(img_dir, base + '.jpg')
        if not os.path.exists(img_path):
            print(f"⚠️  No image for {label_file} (looking for {img_path})")
            continue

        # 4) make the destination dir, preserving sub-folders if you like:
        dest_dir = os.path.join(out_base, label, rel_dir)
        os.makedirs(dest_dir, exist_ok=True)

        # 5) copy or move
        shutil.copy2(img_path, dest_dir)
        #–– if you really want to remove the original, use:
        # shutil.move(img_path, dest_dir)

        print(f"✅  {base}.jpg → {os.path.join(dest_dir, base + '.jpg')}")


In [ ]:
import os
import shutil

# your two parallel trees:
path_images = '/content/drive/MyDrive/DDD-Dataset/data2/Images'
path_labels = '/content/drive/MyDrive/DDD-Dataset/data2/labels'

# where to drop the sorted files:
out_base = '/content/drive/MyDrive/DDD-Dataset/sorted'
out0 = os.path.join(out_base, '0')
out1 = os.path.join(out_base, '1')
os.makedirs(out0, exist_ok=True)
os.makedirs(out1, exist_ok=True)

for dirpath, _, filenames in os.walk(path_labels):
    for fname in filenames:
        if not fname.lower().endswith('.txt'):
            continue

        # 1) read label
        txt_path = os.path.join(dirpath, fname)
        with open(txt_path, 'r') as f:
            raw = f.readline().strip()
        label = raw[0] if raw and raw[0] in ('0','1') else None
        if label is None:
            print(f"⚠️  Skipping {txt_path}: bad label “{raw}”")
            continue

        # 2) build parallel image path
        rel = os.path.relpath(dirpath, path_labels)
        img_name = os.path.splitext(fname)[0] + '.jpg'
        img_path = os.path.join(path_images, rel, img_name)
        if not os.path.exists(img_path):
            print(f"⚠️  No image for {txt_path} (tried {img_path})")
            continue

        # 3) pick destination folder
        dest = out0 if label == '0' else out1

        # 4) copy both .txt and .jpg
        shutil.copy2(txt_path, dest)
        shutil.copy2(img_path, dest)

        print(f"✅  Copied {fname} & {img_name} → {dest}")

In [ ]:
import os
import shutil

# –– adjust this to wherever your “0” folder lives:
src_root = '/content/drive/MyDrive/DDD-Dataset/sorted_images/1'

# –– make a flat “images” folder inside it:
dest_dir = os.path.join(src_root, 'images')
os.makedirs(dest_dir, exist_ok=True)

# –– walk everything under src_root
for dirpath, _, filenames in os.walk(src_root):
    # skip the dest_dir itself so you don’t re-move your moved files
    if os.path.abspath(dirpath) == os.path.abspath(dest_dir):
        continue

    for fname in filenames:
        # filter for whatever image extensions you use
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            src_path = os.path.join(dirpath, fname)
            dst_path = os.path.join(dest_dir, fname)

            # if you want to move:
            shutil.move(src_path, dst_path)
            # or to copy instead, use:
            # shutil.copy2(src_path, dst_path)

            print(f"{src_path!r} → {dst_path!r}")